# Baseline Optimising Realized Volatility Prediction

ดัดแปลงจาก : https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data/comments

<b>Weighted Average Price (WAP)</b> เป็นค่าชี้วัดราคาและปริมาณการซื้อขายของหุ้น ในที่นี้เราจะใช้ WAP เป็น source of stock valuation เพื่อใช้หาค่า realized volatility (ในที่นี้ก็คือ target นั่นเอง) <br> WAP = (bid_price_1*ark_size_1 + ask_price_1*bid_size_1) / (bid_size_1 + ask_size_1)

<b>Log Return</b> เป็นการเปรียบเทียบผลต่างของผลตอบแทน จากรูปแบบผลต่างร้อยละ เช่น ซื้อหุ้นราคา 100 มา 10 หุ้น เมื่อราคาปรับขึ้นเป็น 102 จะทำให้ได้กำไร 10*2 = 20 แต่ถ้าซื้อหุ้นราคา 10 มาจำนวน 100 หุ้น เมื่อราคาปรับขึ้นเป็น 11 จะทำให้ได้กำไร 1*100 = 100 ซึ่งกำไรมันมูลค่าเกินตัวหุ้น ทำให้เราเปรียบเทียบยาก เราจึงทำ stock return ขึ้นมา โดย (102-100)/100 = 2% และ (11-10)/10 = 10% <br> ในที่นี้ เราเอาผลต่างนี้มาทำ log return เพื่อ scale ค่าให้มันกระจายตัวดีขึ้น โดย log retun ระหว่าง t_1 และ t_2 จะเท่ากับ <br> r_(t_1,t_2)=log(S_(t_2)/S_(t_1)) <br> โดย t = t-10min และในสมการเราจะใช้ WAP แทนค่า S

<b>Realized Volatility</b> (𝜎) = sqrt(sum(log return)^2)

โดยในที่นี้ เราจะทำการ Optmise rv จากค่า sec bucket ใน book

In [86]:
import pandas as pd
import numpy as np
import plotly.express as px
from tqdm.auto import tqdm

train = pd.read_csv('optiver-realized-volatility-prediction/train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [62]:
book_example = pd.read_parquet('optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
trade_example = pd.read_parquet('optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

In [63]:
book_example.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [64]:
trade_example.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,21,1.002301,326,12,0
1,5,46,1.002778,128,4,0
2,5,50,1.002818,55,1,0
3,5,57,1.003155,121,5,0
4,5,68,1.003646,4,1,0


In [76]:
book_example['wap'] = (book_example['bid_price1'] * book_example['ask_size1'] +
                                book_example['ask_price1'] * book_example['bid_size1']) / (
                                       book_example['bid_size1']+ book_example['ask_size1'])

In [66]:
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [67]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [68]:
book_example.loc[:,'log_return'] = log_return(book_example['wap'])
book_example = book_example[~book_example['log_return'].isnull()]

In [69]:
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

In [70]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

Realized volatility for stock_id 0 on time_id 5 is 0.004499364172786558


## Naive prediction: using past realized volatility as target

In [17]:
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('optiver-realized-volatility-prediction/book_train.parquet/*')

In [19]:
# def realized_volatility_per_time_id(file_path, prediction_column_name):
#     df_book_data = pd.read_parquet(file_path)
#     df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])/(df_book_data['bid_size1']+ df_book_data['ask_size1'])
    
#     df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
#     df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
#     df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index() 
#     #จากตรงนี้จะได้ค่า real vol ของแต่ละ time id โดยเราจะทำการ reset index เพื่อให้ log_return เป็นเวกเตอร์ของค่าที่ได้จาก agg  
#     df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
#     stock_id = file_path.split('=')[1]
#     df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
#     return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [54]:
# Extention : optimize rv

def opt_realized_volatility_per_time_id(file_path, prediction_column_name, sec):
    df_book_data = pd.read_parquet(file_path)
    df_book_data = df_book_data[df_book_data['seconds_in_bucket'] >= sec] ##
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])/(df_book_data['bid_size1']+ df_book_data['ask_size1'])
    
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index() 
    #จากตรงนี้จะได้ค่า real vol ของแต่ละ time id โดยเราจะทำการ reset index เพื่อให้ log_return เป็นเวกเตอร์ของค่าที่ได้จาก agg  
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

# def past_realized_volatility_per_stock(list_file,prediction_column_name):
#     df_past_realized = pd.DataFrame()
#     for file in tqdm(list_file):
#         df_past_realized = pd.concat([df_past_realized,
#                                      opt_realized_volatility_per_time_id(file_path=file,prediction_column_name=prediction_column_name)])
#     return df_past_realized

# df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train, 
#                                                             prediction_column_name='pred')

# df_past_realized_train.head()

  0%|          | 0/112 [00:00<?, ?it/s]

,row_id,pred
0,17-5,0.002570
1,17-11,0.001350
2,17-16,0.001622
3,17-31,0.001199
4,17-62,0.001287


In [51]:
# def past_realized_volatility_per_stock(list_file,prediction_column_name):
#     df_past_realized = pd.DataFrame()
#     for file in list_file:
#         df_past_realized = pd.concat([df_past_realized,
#                                      realized_volatility_per_time_id(file,prediction_column_name)])
#     return df_past_realized

# df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train, 
#                                                             prediction_column_name='pred')

In [98]:
# df_past_realized_train.head()

In [71]:
# train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
# train = train[['row_id','target']]
# df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')

In [99]:
# df_joined.head()

In [73]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
# R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
# RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
# print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')

Performance of the naive prediction: R2 score: 0.41, RMSPE: 0.447


In [88]:
# Extention : optimize rv

def past_realized_volatility_per_stock(list_file,prediction_column_name,sec=500):
    df_past_realized = pd.DataFrame()
    for file in tqdm(list_file):
        df_past_realized = pd.concat([df_past_realized,
                                     opt_realized_volatility_per_time_id(file_path=file,prediction_column_name=prediction_column_name,sec=sec)])
    return df_past_realized

# optimizing rv loop
best_rmspe = np.inf
for sec in tqdm(np.arange(0, 600, 200), leave = False):
    df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train, 
                                                                prediction_column_name='pred',sec=sec)

    train_opt = train
    train_opt['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    train_opt = train_opt[['row_id','target']]
    df_joined_opt = train_opt.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')

    R2_ = round(r2_score(y_true = df_joined_opt['target'], y_pred = df_joined_opt['pred']),3)
    RMSPE_ = round(rmspe(y_true = df_joined_opt['target'], y_pred = df_joined_opt['pred']),3)
    print(f'sec bucket: {sec}, Performance of the naive prediction: R2 score: {R2_}, RMSPE: {RMSPE_}')
    
    if RMSPE_ < best_rmspe:
            best_rmspe = RMSPE_
            best_sec = sec # cheeze (param that we will use to optimize next)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

sec bucket: 0, Performance of the naive prediction: R2 score: 0.628, RMSPE: 0.341


  0%|          | 0/112 [00:00<?, ?it/s]

sec bucket: 200, Performance of the naive prediction: R2 score: 0.74, RMSPE: 0.287


  0%|          | 0/112 [00:00<?, ?it/s]

sec bucket: 400, Performance of the naive prediction: R2 score: 0.41, RMSPE: 0.447


In [89]:
best_rmspe, best_sec

(0.287, 200)

best_rmspe ดังกล่าวได้มาจากการ loop เพื่อหา sec จากข้อมูล book file ของ stock ทั้งหมด(พร้อมกัน) ซึ่งหากต้องการ optimize มากกว่านี้ แนวทางนึงก็คือการ loop ปรับเทียบ sec สำหรับในแต่ละ stock (แยกหา sec ที่เหมาะสมของแต่ละ stock)